In [5]:
suppressMessages({
    library(SingleCellExperiment)
    library(scRNAseq)
    library(matrixStats)
    library(magrittr)
    library(edgeR)
    library(scran)
})

In [6]:
get_dataset <- function(dataset){
    fname <- sprintf('Datasets/%s_edgeR.RData', dataset)
    dge <- readRDS(fname)
    
    return (dge)
}

get_cell_labels <- function(dataset){
    fname <- sprintf('Datasets/%s-labels.tsv', dataset)
    cell_labels <- read.table(fname, sep='\t', header=1, row=1)
    
    return (cell_labels)
}

get_weighted_vars <- function(dge, ctype1, ctype2, column='group'){
    ctypes <- dge$samples[, column]
    num1 = sum(ctypes %in% ctype1)
    num2 = sum(ctypes %in% ctype2)
    weights = rep(num1, length(ctypes))
    weights[ctypes %in% ctype1] = num2

    wvars <- rowWeightedVars(cpm(dge) %>% log1p, w=weights)
    
    return (wvars)
}

calc_weighted_vars <- function(dge, ctype1, ctype2, column='group'){
    ctypes <- dge$samples[,column]
    count_mat <- cpm(dge)
    weights = rep(0, length(ctypes))
    for (ctype in unique(ctypes)){
        weights[ctypes==ctype] = 1 / sum(ctypes==ctype)
    }
    weights <- round(weights / min(weights))
    
    wvars <- rowWeightedVars(count_mat %>% log1p, w=weights)
    
    return (wvars)
}

filter_features <- function(dge, ctype1, ctype2, column='group'){
    targets = dge$samples[,column] %in% c(ctype1, ctype2)
    dge <- dge[,targets]
    
    keep1 <- rowSums(cpm(dge)>15)> 4
    keep2 <- aveLogCPM(dge) > 4
    keep <- keep1 & keep2
    dge <- dge[keep,,keep.lib.sizes=FALSE]
    vars <- get_weighted_vars(dge, ctype1, ctype2, column=column)
    names(vars) <- rownames(dge)
    vars <- sort(vars, decreasing = TRUE)
    
    return (dge)
}

run_comparison <- function(dge, ctypes1, ctypes2, fname, coef=2){
    # get data of interest
    dge <- filter_features(dge, ctypes1, ctypes2)
    data <- dge$samples
    data$group <- factor(data$group, levels=c(levels(data$group), c('Type_1', 'Type_2')))
    data[data$group %in% ctypes1, 'group'] <- 'Type_1'
    data[data$group %in% ctypes2, 'group'] <- 'Type_2'
    data$group <- factor(data$group, levels=c('Type_1', 'Type_2'))
    data$Batch <- factor(data$Batch)
    
    Group <- data$group
    Batch <- data$Batch
    design <- model.matrix(~Group+Batch, data = data)
    
    # run analysis
    dge <- estimateDisp(dge, design, robust=TRUE)
    fit <- glmQLFit(dge, design, robust=TRUE)
    lrt <- glmQLFTest(fit, coef=coef)
    
    save_edgeR_result(lrt, fname)
}

run_specific_comparison <- function(dge, ctypes1, ctypes2, column, fname){
    dge <- filter_features(dge, ctypes1, ctypes2, column=column)
    data <- dge$samples
    data$group <- data[,column]
    data[data$group %in% ctypes1, 'group'] <- 'Type_1'
    data[data$group %in% ctypes2, 'group'] <- 'Type_2'
    data$group <- factor(data$group, levels=c('Type_1', 'Type_2'))
    
    design <- model.matrix(~data$group, data=data)
    
    # run analysis
    dge <- estimateDisp(dge, design, robust=TRUE)
    #fit <- glmQLFit(dge, design, robust=TRUE)
    #lrt <- glmQLFTest(fit, coef=2)
    lrt<-exactTest(dge, pair=c('NORM', 'HFD'))
    
    save_edgeR_result(lrt, fname)
}

save_edgeR_result <- function(lrt, fname){
    txtname <- sprintf('Results/Differential/edgeR/%s.txt', fname)
    df <- topTags(lrt, n=dim(lrt$table)[1])$table
    df <- df[order(df$PValue), ]
    write.table(df, file=txtname, sep='\t', quote=FALSE)
}

run_all_comparisons <- function(dge, labels, folder){
    dge$samples$group <- labels
    group = as.character(sort(unique(labels)))
    
    for (i in 1:(length(group)-1)){
        for (j in (i+1):length(group)){
            ctype1 <- group[[i]]
            ctype2 <- group[[j]]
            title <- sprintf('%s/%s vs %s', folder, ctype1, ctype2)
            run_comparison(dge, ctype1, ctype2, title)
        }
    }
}

In [7]:
dataset <- 'Lab_Amanda'
dge <- get_dataset(dataset)

fname <- 'NORM vs HFD'
run_specific_comparison(dge, 'NORM', 'HFD', fname, column='CellTypes')
